In [ ]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from scipy import stats

# Get Fama-French data
ff = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1950')[0]
ff = ff/100  # Convert to decimal

# Calculate rolling measures
rolling_window = 60  # 5 years of monthly data
rolling_premium = ff['Mkt-RF'].rolling(rolling_window).mean()
rolling_vol = ff['Mkt-RF'].rolling(rolling_window).std() * np.sqrt(12)  # Annualize
rolling_sharpe = rolling_premium / rolling_vol

# Calculate confidence intervals for market premium
mean_premium = ff['Mkt-RF'].mean() * 12  # Annualized
std_error = ff['Mkt-RF'].std() / np.sqrt(len(ff)) * np.sqrt(12)
conf_interval = stats.t.interval(0.95, len(ff)-1, mean_premium, std_error)

print("Market Risk Premium Analysis")
print(f"Average Annual Premium: {mean_premium:.2%}")
print(f"95% Confidence Interval: ({conf_interval[0]:.2%}, {conf_interval[1]:.2%})")

# Test if premium is significantly different from various levels
test_premium = 0.04  # Test if premium is different from 4%
t_stat = (mean_premium - test_premium) / std_error
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), len(ff)-1))
print(f"\nTest if premium differs from {test_premium:.0%}")
print(f"t-statistic: {t_stat:.2f}")
print(f"p-value: {p_value:.4f}")